In [4]:
pip install BeautifulSoup4

     |████████████████████████████████| 112kB 7.3MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
import numpy as np 
import pandas as pd
import requests 
from bs4 import BeautifulSoup 

print('Libraries imported.')

Libraries imported.


In [20]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 

Page Scrapped.


In [24]:
postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1

Data Collected.


In [27]:
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 
neighbors = pd.DataFrame(columns=column_names)
neighbors

,PostalCode,Borough,Neighborhood


In [30]:
for data in range(len(neighborhoods)):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]

    neighbors = neighbors.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name}, ignore_index=True)

neighbors.head(11)

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,M2A\n
1,M3A\n,North York\n,Parkwoods\n
2,M4A\n,North York\n,Victoria Village\n
3,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront\n
4,M6A\n,North York\n,Lawrence Manor / Lawrence Heights\n
5,M7A\n,Downtown Toronto\n,Queen's Park / Ontario Provincial Government\n
6,M8A\n,Not assigned\n,M9A\n
7,M1B\n,Scarborough\n,Malvern / Rouge\n
8,M2B\n,Not assigned\n,M3B\n
9,M4B\n,East York\n,Parkview Hill / Woodbine Gardens\n
